In [ ]:
# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:21.367856Z","iopub.execute_input":"2025-04-11T22:04:21.368286Z","iopub.status.idle":"2025-04-11T22:04:21.377977Z","shell.execute_reply.started":"2025-04-11T22:04:21.368252Z","shell.execute_reply":"2025-04-11T22:04:21.376846Z"}}
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:19:41.351305Z","iopub.execute_input":"2025-04-11T22:19:41.351737Z","iopub.status.idle":"2025-04-11T22:19:41.356429Z","shell.execute_reply.started":"2025-04-11T22:19:41.351703Z","shell.execute_reply":"2025-04-11T22:19:41.355262Z"}}
import warnings
warnings.filterwarnings("ignore")

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:21.407596Z","iopub.execute_input":"2025-04-11T22:04:21.407968Z","iopub.status.idle":"2025-04-11T22:04:21.448788Z","shell.execute_reply.started":"2025-04-11T22:04:21.407936Z","shell.execute_reply":"2025-04-11T22:04:21.447637Z"}}
train_data=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

print('setup complete')

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:21.450192Z","iopub.execute_input":"2025-04-11T22:04:21.450493Z","iopub.status.idle":"2025-04-11T22:04:21.533136Z","shell.execute_reply.started":"2025-04-11T22:04:21.450466Z","shell.execute_reply":"2025-04-11T22:04:21.532018Z"}}
train_data.describe()

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:21.535045Z","iopub.execute_input":"2025-04-11T22:04:21.535454Z","iopub.status.idle":"2025-04-11T22:04:21.541719Z","shell.execute_reply.started":"2025-04-11T22:04:21.535424Z","shell.execute_reply":"2025-04-11T22:04:21.540719Z"}}
#ვნახოთ ყველა სვეტი რომ გავიგოთ რა არის ჩვენი target variable
train_data.columns

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:21.542887Z","iopub.execute_input":"2025-04-11T22:04:21.543285Z","iopub.status.idle":"2025-04-11T22:04:21.563700Z","shell.execute_reply.started":"2025-04-11T22:04:21.543246Z","shell.execute_reply":"2025-04-11T22:04:21.562488Z"}}
#target variable
y=train_data.SalePrice
train_data=train_data.drop(['SalePrice','Id'],axis=1)
test_data=test_data.drop('Id',axis=1)

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:04:21.564976Z","iopub.execute_input":"2025-04-11T22:04:21.565349Z","iopub.status.idle":"2025-04-11T22:04:21.586349Z","shell.execute_reply.started":"2025-04-11T22:04:21.565309Z","shell.execute_reply":"2025-04-11T22:04:21.585243Z"},"jupyter":{"outputs_hidden":false}}
#გავყოთ data set რადგან მონაცემების დამუშავების შემდეგ გაგვიმარტივდეს მოდელების ვალიდაცია
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train_data, y, test_size=0.2, random_state=42)

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:21.655161Z","iopub.execute_input":"2025-04-11T22:04:21.655506Z","iopub.status.idle":"2025-04-11T22:04:21.665199Z","shell.execute_reply.started":"2025-04-11T22:04:21.655477Z","shell.execute_reply":"2025-04-11T22:04:21.664139Z"}}
#train_data.info()
#info გვეუბნება რომ ჩვენი მონაცემები შეიცავს float, int და object ტიპის სვეტებს

#იმისთვის რომ მოვამზადოთ გასაწვრთნელი dataset გავაცალკევოთ რიცხვითი და ობიქეტის ტიპის მონაცემები
X_tr_numeric=X_tr.select_dtypes(include=['float64','int64'])
X_tr_object=X_tr.select_dtypes(include=['object'])

X_val_numeric=X_val.select_dtypes(include=['float64','int64'])
X_val_object=X_val.select_dtypes(include=['object'])

test_data_numeric=test_data.select_dtypes(include=['float64','int64'])
test_data_object=test_data.select_dtypes(include=['object'])

# %% [markdown]
# # **Data Cleaning**

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:21.666977Z","iopub.execute_input":"2025-04-11T22:04:21.667428Z","iopub.status.idle":"2025-04-11T22:04:21.744184Z","shell.execute_reply.started":"2025-04-11T22:04:21.667385Z","shell.execute_reply":"2025-04-11T22:04:21.742983Z"}}
#გადავაგდოთ ისეთი სვეტები სადაც ძალიან დიდი ინფორმაცია დაკარგულია
def drop_missing(train_data):
    cols_with_missing = [col for col in train_data.columns
                         if train_data[col].isnull().any()]
    
    to_remove=[]
    for col in cols_with_missing:
        null_percentile=(train_data[col].isnull().mean())*100
        if null_percentile>80: to_remove.append(col)
    train_data=train_data.drop(to_remove,axis=1)
    print('Dropping: ',to_remove)
    return train_data

X_tr_numeric=drop_missing(X_tr_numeric)
X_tr_object=drop_missing(X_tr_object)

X_val_numeric=drop_missing(X_val_numeric)
X_val_object=drop_missing(X_val_object)

test_data_numeric=drop_missing(test_data_numeric)
test_data_object=drop_missing(test_data_object)

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:04:21.746271Z","iopub.execute_input":"2025-04-11T22:04:21.746574Z","iopub.status.idle":"2025-04-11T22:04:21.906977Z","shell.execute_reply.started":"2025-04-11T22:04:21.746544Z","shell.execute_reply":"2025-04-11T22:04:21.905603Z"},"jupyter":{"outputs_hidden":false}}
#შევავსოთ მონაცემებში დარჩენილი missing value-ები
def fill_in_gaps(train_data,dep,dependency=True):
    for col in train_data.columns:
        mode_val = train_data[col].mode()[0]
        train_data.fillna({col:mode_val}, inplace=True)
        if dependency:
            dep.fillna({col:mode_val},inplace=True)
    if dependency:
        return train_data,dep
    else:
        return train_data,_


X_tr_numeric,X_val_numeric=fill_in_gaps(X_tr_numeric,X_val_numeric)
X_tr_object,X_val_object=fill_in_gaps(X_tr_object,X_val_object)

test_data_numeric,_=fill_in_gaps(test_data_numeric,_,False)
test_data_object,_=fill_in_gaps(test_data_object,_,False)

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:21.908577Z","iopub.execute_input":"2025-04-11T22:04:21.908943Z","iopub.status.idle":"2025-04-11T22:04:22.149744Z","shell.execute_reply.started":"2025-04-11T22:04:21.908915Z","shell.execute_reply":"2025-04-11T22:04:22.148670Z"}}
#შევეცადოთ ძალიან ექსტრემალური მნიშვნელობების (outliers) მოშორება
from scipy.stats.mstats import winsorize
def remove_outliers(train_data):
    for col in train_data.columns:
        train_data[col]=winsorize(train_data[col], limits=[0.01, 0.01])

    return train_data

X_tr_numeric=remove_outliers(X_tr_numeric)
X_val_numeric=remove_outliers(X_val_numeric)

X_tr_object=remove_outliers(X_tr_object)
X_val_object=remove_outliers(X_val_object)

test_data_numeric=remove_outliers(test_data_numeric)
test_data_object=remove_outliers(test_data_object)

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:22.150857Z","iopub.execute_input":"2025-04-11T22:04:22.151244Z","iopub.status.idle":"2025-04-11T22:04:22.255282Z","shell.execute_reply.started":"2025-04-11T22:04:22.151207Z","shell.execute_reply":"2025-04-11T22:04:22.254370Z"}}
#ამოვშალოთ ისეთი სვეტები სადაც მნიშვნელობები ძირითადად ერთნაირია
def remove_repeated(train_data,dep,dependency=True):
    THRESHOLD = 0.97
    cols_to_drop = []
    for col in train_data.columns:
        most_common_pct = train_data[col].value_counts(normalize=True).iloc[0]
        
        if most_common_pct >= THRESHOLD:
            cols_to_drop.append(col)
    print('Dropping columns: ',cols_to_drop)
    train_data=train_data.drop(cols_to_drop,axis=1)
    if dependency: 
        dep=dep.drop(cols_to_drop,axis=1)

    if dependency: 
        return train_data,dep
    else:
        return train_data,None

X_tr_numeric,X_val_numeric=remove_repeated(X_tr_numeric,X_val_numeric)
X_tr_object,X_val_object=remove_repeated(X_tr_object,X_val_object)

dummy=test_data_numeric.copy()
test_data_numeric,dummy=remove_repeated(test_data_numeric,dummy,False)
test_data_object,dummy=remove_repeated(test_data_object,dummy,False)

# %% [markdown]
# # **Feature Engineering**

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:04:22.256262Z","iopub.execute_input":"2025-04-11T22:04:22.256584Z","iopub.status.idle":"2025-04-11T22:04:22.703318Z","shell.execute_reply.started":"2025-04-11T22:04:22.256559Z","shell.execute_reply":"2025-04-11T22:04:22.702011Z"},"jupyter":{"outputs_hidden":false}}
#დარჩენილ კატეგორიულ მონაცემებს გავუკეთოთ ენკოდირება
from category_encoders.woe import WOEEncoder

binaryColumns = [col for col in X_tr_object.columns if X_tr_object[col].nunique() <= 2]
nonBinaryColumns = [col for col in X_tr_object.columns if X_tr_object[col].nunique() > 2]

median = y_tr.median()
y_ch = (y_tr >= median).astype(int)

woe_encoder = WOEEncoder(cols=nonBinaryColumns)
X_tr_object[nonBinaryColumns] = woe_encoder.fit_transform(X_tr_object[nonBinaryColumns], y_ch)
X_val_object[nonBinaryColumns] = woe_encoder.transform(X_val_object[nonBinaryColumns])
test_data_object[nonBinaryColumns] = woe_encoder.transform(test_data_object[nonBinaryColumns])

X_tr_ohe = pd.get_dummies(X_tr_object[binaryColumns], drop_first=True, dtype=int)
X_val_ohe = pd.get_dummies(X_val_object[binaryColumns], drop_first=True, dtype=int)
X_test_ohe = pd.get_dummies(test_data_object[binaryColumns], drop_first=True, dtype=int)

missing_cols_val = set(X_tr_ohe.columns) - set(X_val_ohe.columns)
for col in missing_cols_val:
    X_val_ohe[col] = 0
X_val_ohe = X_val_ohe[X_tr_ohe.columns]

missing_cols_test = set(X_tr_ohe.columns) - set(X_test_ohe.columns)
for col in missing_cols_test:
    X_test_ohe[col] = 0
X_test_ohe = X_test_ohe[X_tr_ohe.columns]

X_tr_object = X_tr_object.drop(binaryColumns, axis=1)
X_val_object = X_val_object.drop(binaryColumns, axis=1)
test_data_object = test_data_object.drop(binaryColumns, axis=1)

X_tr_object = pd.concat([X_tr_object, X_tr_ohe], axis=1)
X_val_object = pd.concat([X_val_object, X_val_ohe], axis=1)
test_data_object = pd.concat([test_data_object, X_test_ohe], axis=1)

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:04:22.704474Z","iopub.execute_input":"2025-04-11T22:04:22.704895Z","iopub.status.idle":"2025-04-11T22:04:22.710052Z","shell.execute_reply.started":"2025-04-11T22:04:22.704818Z","shell.execute_reply":"2025-04-11T22:04:22.708780Z"},"jupyter":{"outputs_hidden":false}}
assert len(X_tr_object.columns)==len(X_val_object.columns), 'Column quantities dont match'

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:22.713219Z","iopub.execute_input":"2025-04-11T22:04:22.713529Z","iopub.status.idle":"2025-04-11T22:04:22.787783Z","shell.execute_reply.started":"2025-04-11T22:04:22.713502Z","shell.execute_reply":"2025-04-11T22:04:22.786365Z"}}
#ახლა როცა ყველა მონაცემი რიცხვითია, დავთვალოთ სვეტებს შორის კორელაციები და მაღალი კორელაციის მქონე სვეტებიდან გადავაგდოთ ერთ-ერთი
def drop_correlated(X,dep,dependency=True):
    corr_matrix_abs = X.corr().abs()
    high_corr_pairs=set()
    
    for i in corr_matrix_abs.columns:
        for j in corr_matrix_abs.columns:
            if i < j and corr_matrix_abs.loc[i, j] >= 0.85:
                    high_corr_pairs.add((i, j))
    
    high_corr_pairs = list(high_corr_pairs)
    to_drop=[]
    for col1,col2 in high_corr_pairs:to_drop.append(col1)
    print(to_drop)
    X=X.drop(to_drop,axis=1)
    if dependency:
        dep=dep.drop(to_drop,axis=1)
    return X,dep

X_tr_numeric,X_val_numeric=drop_correlated(X_tr_numeric,X_val_numeric)
X_tr_object,X_val_object=drop_correlated(X_tr_object,X_val_object)

test_data_numeric,_=drop_correlated(test_data_numeric,_,False)
test_data_object,_=drop_correlated(test_data_object,_,False)

# %% [markdown] {"jupyter":{"outputs_hidden":false}}
# # **Training**

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:22.789897Z","iopub.execute_input":"2025-04-11T22:04:22.790252Z","iopub.status.idle":"2025-04-11T22:04:22.803947Z","shell.execute_reply.started":"2025-04-11T22:04:22.790218Z","shell.execute_reply":"2025-04-11T22:04:22.802903Z"}}
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.linear_model import LinearRegression


X_tr = pd.concat(
    [X_tr_numeric, X_tr_object], 
    axis=1,
    verify_integrity=True
)
X_val = pd.concat(
    [X_val_numeric, X_val_object], 
    axis=1,
    verify_integrity=True
)

test_data = pd.concat(
    [test_data_numeric, test_data_object], 
    axis=1,
    verify_integrity=True
)

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:04:22.805072Z","iopub.execute_input":"2025-04-11T22:04:22.805403Z","iopub.status.idle":"2025-04-11T22:04:22.813146Z","shell.execute_reply.started":"2025-04-11T22:04:22.805376Z","shell.execute_reply":"2025-04-11T22:04:22.811902Z"}}
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold,GridSearchCV
from sklearn.metrics import r2_score, mean_squared_log_error
import numpy as np
from sklearn.preprocessing import StandardScaler

kf = KFold(n_splits=5, shuffle=True, random_state=42)

results = {}

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:04:22.814215Z","iopub.execute_input":"2025-04-11T22:04:22.814513Z","iopub.status.idle":"2025-04-11T22:04:22.891710Z","shell.execute_reply.started":"2025-04-11T22:04:22.814481Z","shell.execute_reply":"2025-04-11T22:04:22.890511Z"},"jupyter":{"outputs_hidden":false}}
print("Testing Linear Regression...")
lr = LinearRegression()
lr_scores = cross_val_score(lr, X_tr, y_tr, cv=kf, scoring='r2')
results['Linear'] = {
    'CV R2 Mean': np.mean(lr_scores),
    'CV R2 Std': np.std(lr_scores)
}

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:04:22.892813Z","iopub.execute_input":"2025-04-11T22:04:22.893232Z","iopub.status.idle":"2025-04-11T22:05:11.823276Z","shell.execute_reply.started":"2025-04-11T22:04:22.893191Z","shell.execute_reply":"2025-04-11T22:05:11.821705Z"},"jupyter":{"outputs_hidden":false}}
print("Testing Random Forest...")
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
rf = RandomForestRegressor(random_state=42)
rf_grid = GridSearchCV(rf, rf_params, cv=kf, scoring='r2', n_jobs=-1)
rf_grid.fit(X_tr, y_tr)
results['RandomForest'] = {
    'CV R2 Mean': rf_grid.best_score_,
    'Best Params': rf_grid.best_params_
}

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:05:11.824870Z","iopub.execute_input":"2025-04-11T22:05:11.825337Z","iopub.status.idle":"2025-04-11T22:05:28.788735Z","shell.execute_reply.started":"2025-04-11T22:05:11.825292Z","shell.execute_reply":"2025-04-11T22:05:28.787752Z"},"jupyter":{"outputs_hidden":false}}
print("Testing XGBoost...")
xgb_params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 0.9]
}
xgb = XGBRegressor(random_state=42)
xgb_grid = GridSearchCV(xgb, xgb_params, cv=kf, scoring='r2', n_jobs=-1)
xgb_grid.fit(X_tr, y_tr)
results['XGBoost'] = {
    'CV R2 Mean': xgb_grid.best_score_,
    'Best Params': xgb_grid.best_params_
}

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:05:28.789917Z","iopub.execute_input":"2025-04-11T22:05:28.790706Z","iopub.status.idle":"2025-04-11T22:05:29.729412Z","shell.execute_reply.started":"2025-04-11T22:05:28.790670Z","shell.execute_reply":"2025-04-11T22:05:29.728334Z"},"jupyter":{"outputs_hidden":false}}
print("Testing Lasso...")
lasso_params = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]}
lasso = Lasso(random_state=42)
lasso_grid = GridSearchCV(lasso, lasso_params, cv=kf, scoring='r2', n_jobs=-1)
lasso_grid.fit(X_tr, y_tr)
results['Lasso'] = {
    'CV R2 Mean': lasso_grid.best_score_,
    'Best Params': lasso_grid.best_params_
}

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:05:29.730488Z","iopub.execute_input":"2025-04-11T22:05:29.730863Z","iopub.status.idle":"2025-04-11T22:05:29.858776Z","shell.execute_reply.started":"2025-04-11T22:05:29.730822Z","shell.execute_reply":"2025-04-11T22:05:29.857899Z"},"jupyter":{"outputs_hidden":false}}
print("Testing Ridge...")
ridge_params = {'alpha': [0.1, 1, 10, 100, 1000]}
ridge = Ridge(random_state=42)
ridge_grid = GridSearchCV(ridge, ridge_params, cv=kf, scoring='r2', n_jobs=-1)
ridge_grid.fit(X_tr, y_tr)
results['Ridge'] = {
    'CV R2 Mean': ridge_grid.best_score_,
    'Best Params': ridge_grid.best_params_
}

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:05:29.859487Z","iopub.execute_input":"2025-04-11T22:05:29.860032Z","iopub.status.idle":"2025-04-11T22:05:29.867522Z","shell.execute_reply.started":"2025-04-11T22:05:29.859990Z","shell.execute_reply":"2025-04-11T22:05:29.866126Z"},"jupyter":{"outputs_hidden":false}}
best_models = {
    'LinearRegression':lr,
    'RandomForest': rf_grid.best_estimator_,
    'XGBoost': xgb_grid.best_estimator_,
    'Lasso': lasso_grid.best_estimator_,
    'Ridge': ridge_grid.best_estimator_
}

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:05:29.868754Z","iopub.execute_input":"2025-04-11T22:05:29.869148Z","iopub.status.idle":"2025-04-11T22:05:33.714347Z","shell.execute_reply.started":"2025-04-11T22:05:29.869107Z","shell.execute_reply":"2025-04-11T22:05:33.713213Z"},"jupyter":{"outputs_hidden":false}}
print("Validation on Best Models:")
for name, model in best_models.items():
    model.fit(X_tr, y_tr)
    preds = model.predict(X_val)
    r2 = r2_score(y_val, preds)
    rmse = np.sqrt(mean_squared_log_error(y_val, preds))
    print(f"{name}:")
    print(f"  Validation R2: {r2:.4f}")
    print(f"  Validation RMSLE: {rmse:.2f}")

# %% [markdown] {"jupyter":{"outputs_hidden":false}}
# **საუკეთესო მოდელად შეიძლება ჩაითვალოს XGBoost, თუმცა სხვა მოდელებსაც მისაღები შედეგები აქვთ**

# %% [markdown]
# # **ML Flow Tracking**

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:05:33.715383Z","iopub.execute_input":"2025-04-11T22:05:33.715666Z","iopub.status.idle":"2025-04-11T22:05:38.295166Z","shell.execute_reply.started":"2025-04-11T22:05:33.715641Z","shell.execute_reply":"2025-04-11T22:05:38.293881Z"}}
!pip install dagshub

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:05:38.296537Z","iopub.execute_input":"2025-04-11T22:05:38.296929Z","iopub.status.idle":"2025-04-11T22:05:38.495117Z","shell.execute_reply.started":"2025-04-11T22:05:38.296898Z","shell.execute_reply":"2025-04-11T22:05:38.493846Z"}}
import dagshub
dagshub.init(repo_owner='skara-21', repo_name='Assignment1_ARD', mlflow=True)

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:05:38.496219Z","iopub.execute_input":"2025-04-11T22:05:38.496694Z","iopub.status.idle":"2025-04-11T22:05:43.195740Z","shell.execute_reply.started":"2025-04-11T22:05:38.496655Z","shell.execute_reply":"2025-04-11T22:05:43.194439Z"}}
!pip install mlflow

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2025-04-11T22:05:43.197192Z","iopub.execute_input":"2025-04-11T22:05:43.197566Z","iopub.status.idle":"2025-04-11T22:06:50.421019Z","shell.execute_reply.started":"2025-04-11T22:05:43.197535Z","shell.execute_reply":"2025-04-11T22:06:50.419888Z"}}
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score, mean_squared_log_error
from scipy import stats
from dagshub import dagshub_logger
from mlflow.models.signature import infer_signature

mlflow.set_tracking_uri('https://dagshub.com/skara-21/Assignment1_ARD.mlflow')
mlflow.set_experiment("House_Price_Prediction_Enhanced")

def adjusted_r2(r2, n, p):
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

def calculate_f_statistic(r2, n, p):
    return (r2 / p) / ((1 - r2) / (n - p - 1))

def create_input_example(X, n_samples=5):
    if len(X) > n_samples:
        sample = X.sample(n_samples, random_state=42)
    else:
        sample = X.copy()
    return sample

def log_model_performance(model, X_tr, y_tr, X_val, y_val, name):
    with mlflow.start_run(nested=True, run_name=name):
        model.fit(X_tr, y_tr)

        input_example = create_input_example(X_tr)
        signature = infer_signature(X_tr, model.predict(X_tr))
        
        train_preds = model.predict(X_tr)
        val_preds = model.predict(X_val)
        
        n = X_tr.shape[0]
        p = X_tr.shape[1]
        
        train_r2 = r2_score(y_tr, train_preds)
        train_adj_r2 = adjusted_r2(train_r2, n, p)
        train_rmse = np.sqrt(mean_squared_error(y_tr, train_preds))
        train_mae = mean_absolute_error(y_tr, train_preds)
        train_msle = mean_squared_log_error(y_tr, train_preds)
        
        val_r2 = r2_score(y_val, val_preds)
        val_adj_r2 = adjusted_r2(val_r2, len(y_val), p)
        val_rmse = np.sqrt(mean_squared_error(y_val, val_preds))
        val_mae = mean_absolute_error(y_val, val_preds)
        val_msle = mean_squared_log_error(y_val, val_preds)
        
        f_stat = None
        if hasattr(model, 'coef_'):
            f_stat = calculate_f_statistic(train_r2, n, p)
            p_value = stats.f.sf(f_stat, p, n-p-1)
        
        cv_scores = None
        if hasattr(model, 'best_score_'):
            cv_scores = model.best_score_
        
        if hasattr(model, 'best_params_'):
            mlflow.log_params(model.best_params_)
        elif hasattr(model, 'get_params'):
            mlflow.log_params(model.get_params())
        
        mlflow.log_metrics({
            "train_r2": train_r2,
            "train_adj_r2": train_adj_r2,
            "train_rmse": train_rmse,
            "train_mae": train_mae,
            "train_msle": train_msle
        })
        
        mlflow.log_metrics({
            "val_r2": val_r2,
            "val_adj_r2": val_adj_r2,
            "val_rmse": val_rmse,
            "val_mae": val_mae,
            "val_msle": val_msle
        })
        
        if f_stat is not None:
            mlflow.log_metrics({
                "f_statistic": f_stat,
                "f_p_value": p_value
            })
        
        if cv_scores is not None:
            mlflow.log_metrics({
                "cv_mean_score": cv_scores
            })
        
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path=name,
            signature=signature,
            input_example=input_example
        )
        
        print(f"{name}")
        print(f"Training R2: {train_r2:.4f}")
        print(f"Validation R2: {val_r2:.4f}")
        print(f"Validation RMSE: {val_rmse:.4f}")
        print(f"MAE: {val_mae:.4f}")
        print(f"MSLE: {val_msle:.4f}")
        if f_stat:
            print(f"F-statistic: {f_stat:.2f}")
            print(f"p-value: {p_value:.4f}")
        if cv_scores:
            print(f"CV Mean Score: {cv_scores:.4f}")

with dagshub_logger() as logger:
    logger.log_hyperparams({
        "train_shape": X_tr.shape,
        "validation_shape": X_val.shape,
        "n_features": X_tr.shape[1]
    })
    
    for name, model in best_models.items():
        log_model_performance(model, X_tr, y_tr, X_val, y_val, name)


# %% [markdown]
# # **Result saving**

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:11:43.516879Z","iopub.execute_input":"2025-04-11T22:11:43.517283Z","iopub.status.idle":"2025-04-11T22:11:43.623545Z","shell.execute_reply.started":"2025-04-11T22:11:43.517250Z","shell.execute_reply":"2025-04-11T22:11:43.622610Z"}}
test_data.to_csv('/kaggle/working/processed_test_data.csv', index=False)

# %% [code] {"execution":{"iopub.status.busy":"2025-04-11T22:19:47.169067Z","iopub.execute_input":"2025-04-11T22:19:47.169415Z","iopub.status.idle":"2025-04-11T22:19:54.704579Z","shell.execute_reply.started":"2025-04-11T22:19:47.169388Z","shell.execute_reply":"2025-04-11T22:19:54.703511Z"}}
mlflow.start_run()

mlflow.log_param("Logistic", "XGBoost")
mlflow.log_metric("Accuracy", 0.91)
mlflow.sklearn.log_model(
    sk_model=best_models['XGBoost'],
    artifact_path="models",
    registered_model_name="BestModel"
)

mlflow.end_run()